In [ ]:
#!/usr/bin/env python
import ROOT, sys, os
from ROOT import std

from larlite import larlite
from larcv import larcv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)
from ROOT import std
from ROOT import geo2d
from ROOT import cv
from ROOT.cv import Point_ as Vector
DTYPE='float'
import matplotlib.patches as patches

In [ ]:
proc = larcv.ProcessDriver("ProcessDriver")
proc.configure('shower_track_unit.fcl')
flist=std.vector('string')()
flist.push_back('nue_example.root')
proc.override_input_file(flist)
proc.initialize()

larbysimg = proc.process_ptr(0)

#proc.batch_process(5,1)
proc.batch_process(7,1)
#proc.batch_process(9,1)
#proc.batch_process(10,1)
#proc.batch_process(11,1)
#proc.batch_process(14,1)
#proc.batch_process(17,1) # for talk
#proc.batch_process(18,1)
#proc.batch_process(20,1)
#proc.batch_process(94,1)# candidate
#proc.batch_process(122,1)# for talk

In [ ]:
#xranges=((450,700),(450,700),(450,700))
#yranges=((875,1125),(700,950),(1350,1600))

xranges=((350,480),(300,476),(300,476))
yranges=((100,200),(0,500),(0,500))

force_range = False
#force_range = True

In [ ]:
pygeo   = geo2d.PyDraw()

mgr=larbysimg.Manager()
img_v = []
shower_img_v = []
track_img_v  = []

track_mat_v  = mgr.InputImages(0)
shower_mat_v = mgr.InputImages(1)

for plane in xrange(track_mat_v.size()):
    shower_img_v.append(pygeo.image(shower_mat_v[plane]))
    track_img_v.append(pygeo.image(track_mat_v[plane]))
    img_v.append(shower_img_v[-1] + track_img_v[-1])
    shower_img=np.where(shower_img_v[plane]>10.0,85.0,0.0).astype(np.uint8)
    track_img=np.where(track_img_v[plane]>10.0,160.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    plt.imshow(shower_img+track_img,cmap='jet',interpolation='none',vmin=0.,vmax=255.)
    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()

dm=mgr.DataManager()

In [ ]:
mgr=larbysimg.Manager()
alg=mgr.GetClusterAlg(9)
vtx2d_vv = alg.ScannedVertex2D()

for plane in xrange(3):
    print 'Plane',plane
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')

    #Get the image
    #shape_img = img_v[plane]
    #shape_img = track_img_v[plane]
    shape_img = shower_img_v[plane]
    shape_img=np.where(shape_img>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    
    for vtx2d_v in vtx2d_vv:
        vtx2d = vtx2d_v[plane]
        pt = vtx2d.center
        radius = vtx2d.radius
        xs_v = vtx2d.xs_v
        
        c=patches.Circle((pt.x,pt.y),radius,ec='cyan',fc='none',lw=5)
        ax.add_patch(c)
        
        plt.plot([pt.x],[pt.y],marker='$\star$',color='blue',markersize=24)
        
        for xs in xs_v:
            plt.plot([xs.pt.x],[xs.pt.y],marker='o',color='magenta',markersize=8)
        
    ax.set_aspect(1.0)
    plt.tight_layout()
    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()
        

In [ ]:
mgr=larbysimg.Manager()
dm=mgr.DataManager()
lintrack_data=dm.Data(8)
sshower_data=dm.Data(9)

track_v  = lintrack_data.get_clusters()
shower_v = sshower_data.get_showers() 
for plane in xrange(3):
    print 'Plane',plane
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')

    #Get the image
    shape_img = img_v[plane]
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    
    for track_id in xrange(track_v.size()):
        track = track_v[track_id]
        edge1=track.edge1
        edge2=track.edge2
        if edge1.vtx2d_v.empty() or edge2.vtx2d_v.empty():
            print 'Track',track_id,'has one edge'
        else:
            print 'Track',track_id,'has two edge'
        strack = track.get_cluster(plane)
        if strack.ctor.empty(): continue
            
        pts=np.array([[pt.x,pt.y] for pt in strack.ctor])
        plt.plot(pts[:,0],pts[:,1],'-o',color='blue')
        
        plt.plot([strack.edge1.x],[strack.edge1.y],marker='$\star$',color='magenta',markersize=24)
        plt.plot([strack.edge2.x],[strack.edge2.y],marker='$\star$',color='magenta',markersize=24)
        
    for shower_id in xrange(shower_v.size()):
        ctor  = shower_v[shower_id].get_cluster(plane).ctor        
        pts=[[pt.x,pt.y] for pt in ctor]
        pts.append(pts[0])
        pts = np.array(pts)
        plt.plot(pts[:,0],pts[:,1],'-o',color='red')
        
        start = shower_v[shower_id].get_cluster(plane).start
        print start.x,start.y
        plt.plot([start.x],[start.y],marker='$\star$',color='yellow',markersize=24)
        
    ax.set_aspect(1.0)
    plt.tight_layout()
    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()
        